# Forecasting weather with LSTM

In this notebook we are forecasting the temp of future years based on the temp of the past and current year.

Based on this source: https://colab.research.google.com/drive/1b3CUJuDOmPmNdZFH3LQDmt5F0K3FZhqD?usp=sharing#scrollTo=1q73lN27SDPC

In [ ]:
# Importing libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# Using a Tensorflow dataset

For this example we are using a Tensorflow dataset. This is optional. You can use your custom numerical datasets instead. 

Source: https://www.tensorflow.org/tutorials/structured_data/time_series

Thorough look into the weather data: https://www.bgc-jena.mpg.de/wetter/

In [ ]:
zip_path = tf.keras.utils.get_file(
        origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
        fname='jena_climate_2009_2016.csv.zip',
        extract=True)
csv_path, _ = os.path.splitext(zip_path)

# Reading and printing the data from the csv file

As you will see in the next code cell, this weather dataset is composed by 420551 rows × 15 columns.

The first column contains the time-related data (date and time) and shows us that we now have access to weather information that were being collected constantly, with 10min intervals (00:10:00, 00:20:00, 00:30:00 and so on), for 8 years (2009 through 2017).

In [ ]:
# df stands for dataframe, which is a two-dimensional data structure,
# i.e., data aligned in a tabular fashion in rows and columns
df = pd.read_csv(csv_path)
df

In [ ]:
# Removing some information and taking 1h intervals between measurements
# Starting from the 5th row (00:00:00) and taking every sixth row after that
df = df[5::6]
df

In [ ]:
# Getting the first column as our index number
df.index = pd.to_datetime(df['Date Time'], format='%d.%m.%Y %H:%M:%S')
df[:10]

In [ ]:
# Getting and printing the recurrent pattern of temperatures through the years/months/days
temp = df['T (degC)']
temp.plot()

# Forecasting as a supervised learning problem

This is where things get interesting. At this step, we define the number of sequential data that define the value of the next one. 

In our case, we will use a window size of 5, which means that every 5 dataframes, we make a prediction for the next one. Then, the predicted one becomes part of the next batch of 5 and the predicted one becomes part of the next batch of 5, and so on.

In [ ]:
# Every 5 hours predict the temp for the next hour:
# ---------------------------------------------------------------
#  X matrix                       y vector (correponsing output)
# ---------------------------------------------------------------
# [[[1], [2], [3], [4], [5]]]    [6]
# [[[2], [3], [4], [5], [6]]]    [7]
# ---------------------------------------------------------------

def df_to_X_y(df, window_size):
    df_as_np = df.to_numpy()
    X = []
    y = []
    for i in range(len(df_as_np)-window_size):
        row = [[a] for a in df_as_np[i:i+window_size]]
        X.append(row)
        label = df_as_np[i+window_size]
        y.append(label)
    return np.array(X), np.array(y)

In [ ]:
# X holds all the batches of 5 for all dataframes
# y holds all the dataframe values that follow every batch of 5
WINDOW_SIZE = 5
X, y = df_to_X_y(temp, WINDOW_SIZE)
X.shape, y.shape
#print(X, y)

In [ ]:
#Spliting dataset in training, validation and testing datasets
X_train, y_train = X[:60000], y[:60000]
X_val, y_val = X[60000:65000], y[60000:65000]
X_test, y_test = X[65000:], y[65000:]
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

# Building our Model

In [ ]:
# Importing libraries for our model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

In [ ]:
# Incorporating an LSTM model with 64 units
# The output is just one number, one prediction
model = Sequential()
model.add(InputLayer((5,1)))
model.add(LSTM(64))
model.add(Dense(8, 'relu'))
model.add(Dense(1, 'linear'))

model.summary()

In [ ]:
# Saving only the one that has the lowest validation loss
cp = ModelCheckpoint('model/', save_best_only=True)
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])

In [ ]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=2, callbacks=[cp]) #10

In [ ]:
# Loading our model
model = load_model('model/')

In [ ]:
# Making some predictions with our training data
# Comparing our predictions with the actual data
train_predictions = model.predict(X_train).flatten()
train_results = pd.DataFrame(data={'Train Predictions':train_predictions, 'Actuals':y_train})
train_results

In [ ]:
# Seeing the difference between them in a graph
plt.plot(train_results['Train Predictions'][:100], 'b-')
plt.plot(train_results['Actuals'][:100], 'r-')

In [ ]:
# Making some predictions with our validation data
# Comparing our predictions with the actual data
val_predictions = model.predict(X_val).flatten()
val_results = pd.DataFrame(data={'Val Predictions':val_predictions, 'Actuals':y_val})
val_results

In [ ]:
# Seeing the difference between them in a graph
plt.plot(val_results['Val Predictions'][:100], 'b-')
plt.plot(val_results['Actuals'][:100], 'r-')

In [ ]:
# Making some predictions with our testing data
# Comparing our predictions with the actual data
test_predictions = model.predict(X_test).flatten()
test_results = pd.DataFrame(data={'Test Predictions':test_predictions, 'Actuals':y_test})
test_results

In [ ]:
# Seeing the difference between them in a graph
plt.plot(test_results['Test Predictions'][:100], 'b-')
plt.plot(test_results['Actuals'][:100], 'r-')